# Execution Simulation

This notebook compares TWAP and VWAP strategies using the event-driven simulator.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from src.data.synthetic import generate_synthetic_lob
from src.impact_models.parametric import AlmgrenChrissModel, ImpactParams
from src.execution.strategies import TWAPStrategy, VWAPStrategy
from src.evaluation.backtest import BacktestRunner
from src.evaluation.metrics import ExecutionMetrics

%matplotlib inline

## 1. Setup Simulation Environment

In [ ]:
# Generate Data
df = generate_synthetic_lob(n_events=10000, volatility=0.3, seed=999)
start_time = df['timestamp'].min()
end_time = df['timestamp'].max()
duration = end_time - start_time

# Impact Model
params = ImpactParams(eta=0.5, gamma=0.01)
model = AlmgrenChrissModel(params)

runner = BacktestRunner(df, model)

## 2. Run TWAP

In [ ]:
twap_params = {
    'total_size': 1000,
    'duration': duration,
    'start_time': start_time,
    'n_slices': 20
}

twap_trades = runner.run(TWAPStrategy, twap_params)
twap_vwap = ExecutionMetrics.calculate_vwap(twap_trades)
print(f"TWAP Executed VWAP: {twap_vwap:.2f}")

## 3. Run VWAP (with flat profile for simplicity)

In [ ]:
vwap_params = {
    'total_size': 1000,
    'duration': duration,
    'start_time': start_time,
    'volume_profile': [1] * 20 # Flat profile
}

vwap_trades = runner.run(VWAPStrategy, vwap_params)
vwap_vwap = ExecutionMetrics.calculate_vwap(vwap_trades)
print(f"VWAP Executed VWAP: {vwap_vwap:.2f}")

## 4. Compare Slippage

In [ ]:
arrival_price = df.iloc[0]['price']
twap_slippage = ExecutionMetrics.calculate_slippage(twap_trades, arrival_price)
vwap_slippage = ExecutionMetrics.calculate_slippage(vwap_trades, arrival_price)

print(f"Arrival Price: {arrival_price:.2f}")
print(f"TWAP Slippage (bps): {twap_slippage:.2f}")
print(f"VWAP Slippage (bps): {vwap_slippage:.2f}")

plt.bar(['TWAP', 'VWAP'], [twap_slippage, vwap_slippage])
plt.ylabel('Slippage (bps)')
plt.title('Execution Strategy Comparison')
plt.show()